In [19]:
import sqlite3 as sq
import pandas as pd

conn = sq.connect('example.db')

In [9]:
df = pd.read_csv('/home/phermes/Desktop/stocklist/companylist.csv')

In [11]:
df2 = pd.read_csv('/home/phermes/Desktop/stocklist/companylist2.csv')

In [17]:
for s in df.Symbol:
    print s

PIH
FLWS
FCCY
SRCE
VNET
TWOU
JOBS
CAFD
EGHT
AVHI
SHLM
AAON
ABAX
ABEO
ABEOW
ABIL
ABMD
AXAS
ACIU
ACIA
ACTG
ACHC
ACAD
ACST
AXDX
ACCP
XLRN
ANCX
ARAY
ACRX
ACET
AKAO
ACHN
ACIW
ACRS
ACNB
ACOR
ATVI
ACTA
ACUR
ACXM
ADMS
ADMP
ADAP
ADUS
AEY
IOTS
ADMA
ADBE
ADTN
ADRO
AAAP
ADES
AEIS
AITP
AMD
ADXS
ADXSW
ADVM
MAUI
AEGN
AGLE
AEHR
AMTX
AEPI
AERI
AVAV
AEZS
AEMD
GNMX
AFMD
AGEN
AGRX
AGYS
AGIO
AGNC
AGNCB
AGNCP
AGFS
AGFSW
AIMT
AIRM
AIRT
ATSG
AIRG
AMCN
AKAM
AKTX
AKBA
AKER
AKRX
ALRM
ALSK
AMRI
ALBO
ABDC
ADHD
ALDR
ALDX
ALXN
ALCO
ALGN
ALIM
ALJJ
ALKS
ABTX
ALGT
AIQ
AHGP
AMMA
ARLP
AHPI
AMOT
ALQA
ALLT
MDRX
AFAM
ALNY
AOSL
GOOG
GOOGL
SMCP
ATEC
SWIN
ASPS
AIMC
AMAG
AMRN
AMRK
AYA
AMZN
AMBC
AMBCW
AMBA
AMCX
DOX
AMDA
AMED
UHAL
ATAX
AMOV
AAL
ACSF
AETI
AMNB
ANAT
AOBC
APEI
ARII
AMRB
AMSWA
AMSC
AMWD
CRMT
ABCB
AMSF
ASRV
ASRVP
ATLO
AMGN
FOLD
AMKR
AMPH
IBUY
ASYS
AFSI
AMRS
ADI
ALOG
AVXL
ANCB
ANDA
ANDAR
ANDAU
ANDAW
ANGI
ANGO
ANIP
ANIK
ANSS
ATRS
ANTH
ABAC
APOG
APOL
APEN
AINV
AMEH
APPF
AAPL
ARCI
APDN
APDNW
AGTC
AMAT
AMCC
AAOI
AREX
APTI


In [20]:
c = conn.cursor()

In [21]:
c.execute('''CREATE TABLE stocks
             (ticker text, name text)''')

In [22]:
c.execute('SELECT * FROM stocks')